# Part 4b - Saving a Model

In this notebook we will cover the following topics:

* Saving a model to disk

In [ ]:
import numpy as np
np.warnings.filterwarnings('ignore')  # Hide np.floating warning

import keras

from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

# Prevent TensorFlow from grabbing all the GPU memory
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

import holoviews as hv
hv.extension('bokeh')

## Load the Data

In [ ]:
from keras.datasets import cifar10
import keras.utils

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Save an unmodified copy of y_test for later, flattened to one column
y_test_true = y_test[:,0].copy()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# The data only has numeric categories so we also have the string labels below 
cifar10_labels = np.array(['airplane', 'automobile', 'bird', 'cat', 'deer', 
                           'dog', 'frog', 'horse', 'ship', 'truck'])

## Train the Model

Let's quickly train our simple model so we can save the results

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

Saving the model is as easy as calling the `save()` method.  This records the weights and the structure of the model so that it can be recreated by another program entirely from this file.  (Assuming that program is using the same version of Keras.)

In [ ]:
model.save('cifar10_model.hdf5')

Note that the file format is HDF5, which is a common data format for numerical data.  Keras requires the `h5py` Python package be present in order to read and write HDF5 files.

Depending on the number of weights in the model, this file can get very big:

In [ ]:
! ls -lh cifar10_model.hdf5

We can poke around to see the structure of it using the HDF5 command line tools:

In [ ]:
! h5ls -r cifar10_model.hdf5

Interestingly, we can see that the HDF5 file also records the state of the optimizer, so that we can resume training from a saved model.  This is a useful way to checkpoint your work.  In fact, Keras has a [ModelCheckpoint callback](https://keras.io/callbacks/#modelcheckpoint) that does this automatically after every epoch.

## Experiments to Try

* Try using the `ModelCheckpoint` callback to save the model in every epoch.

If you screw everything up, you can use File / Revert to Checkpoint to go back to the first version of the notebook and restart the Jupyter kernel with Kernel / Restart.